In [1]:
import dualcodec
import torchaudio
from IPython.display import Audio

/home/vansh/dualcodec-exp/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_id = "12hz_v1"

In [ ]:
path_1 = "output_checkpoints_2/dualcodec_experiments_fully_causal/checkpoint/epoch-0000_step-0094400_loss-135.731720-dualcodec_experiments_fully_causal"

dualcodec_model_1 = dualcodec.get_model_fixed(model_id, path_1)

FileNotFoundError: file output_checkpoints_2/dualcodec_experiments_fully_causal/checkpoint/epoch-0000_step-0091000_loss-269.028168-dualcodec_experiments_fully_causal not found

In [ ]:
dualcodec_inference_1 = dualcodec.Inference(dualcodec_model=dualcodec_model, device="cpu")

In [ ]:
audio, sr = torchaudio.load("audio_samples/tara.wav")
audio = torchaudio.functional.resample(audio, sr, 24000)
audio = audio.reshape(1,1,-1)
audio = audio.cpu()

Audio(audio.squeeze(0), rate=24000)

In [ ]:
semantic_codes, acoustic_codes = dualcodec_inference_1.encode(audio, n_quantizers=8)
out_audio = dualcodec_inference_1.decode(semantic_codes, acoustic_codes)
Audio(out_audio.cpu().squeeze(0), rate=24000)

In [ ]:
path_2 = "output_checkpoints_2/dualcodec_experiments_fully_causal/checkpoint/epoch-0000_step-0083400_loss-169.211517-dualcodec_experiments_fully_causal"

dualcodec_model_2 = dualcodec.get_model(model_id, path_2)

In [ ]:
dualcodec_inference_1 = dualcodec.Inference(dualcodec_model=dualcodec_model_2, device="cpu")

In [ ]:
audio, sr = torchaudio.load("audio_samples/tara.wav")
audio = torchaudio.functional.resample(audio, sr, 24000)
audio = audio.reshape(1,1,-1)
audio = audio.cpu()

Audio(audio.squeeze(0), rate=24000)

In [ ]:
semantic_codes, acoustic_codes = dualcodec_inference_1.encode(audio, n_quantizers=8)
out_audio = dualcodec_inference_1.decode(semantic_codes, acoustic_codes)
Audio(out_audio.cpu().squeeze(0), rate=24000)